In [1]:
!pip3 install geopandas fiona
!pip3 install folium matplotlib mapclassify

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for geopandas from https://files.pythonhosted.org/packages/3e/b0/69fa7a0f55122847506a42fea6988d03b34136938082f142151bc9d9f7e7/geopandas-0.14.4-py3-none-any.whl.metadata
  Obtaining dependency information for fiona from https://files.pythonhosted.org/packages/72/8b/8313d6571c9bc89fa7dce22b78467b3c75415874657710061c518c1ce44b/fiona-1.9.6-cp39-cp39-macosx_11_0_arm64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 665.5 kB/s eta 0:00:00 0:00:01
  Obtaining dependency information for pyproj>=3.3.0 from https://files.pythonhosted.org/packages/2c/c2/8d4f61065dfed965e53badd41201ad86a05af0c1bbc75dffb12ef0f5a7dd/pyproj-3.6.1-cp39-cp39-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for shapely>=1.8.0 from https://files.pythonhosted.org/packages/ed/0f/0d0063a407a6827f2335c53190535c35fa6744a9da7ce4a77f56b984b109/shapely-2.0.4-cp39-cp39-macosx_11_

In [70]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from shapely.geometry import Point


Download the shapefile in https://www.census.gov/cgi-bin/geo/shapefiles/index.php?year=2023&layergroup=Counties+%28and+equivalent%29

In [71]:
counties_gdf = gpd.read_file("tl_2023_us_county/tl_2023_us_county.shp")
counties_gdf = counties_gdf.rename(columns={"GEOID":"fips"})
population_df = pd.read_csv('county_population.csv', dtype={'fips':str}, index_col=0)
## asdf#population_df['log_population'] = np.log10(population_df['population'])

population_year_df = population_df.query('year == 2023')
gdf_population_year = pd.merge(counties_gdf, population_year_df, how='left', on='fips')

In [128]:
len(gdf_population_year)

3235

In [119]:
def coordinate_to_county(long,lat):
    """
    Input:
    lat: latitude
    long: longitude
    
    Output:
    FIPS, county name in 2023, state
    """
    coord = Point(long,lat)
    idx = gdf_population_year['geometry'].contains(coord)
    res = gdf_population_year[idx]
    try:
        return res['fips'].item(), res['state'].item(), res['county'].item()
    except ValueError:
        print(f"Coordinate {long,lat} is not contained in any county")

In [127]:
print(coordinate_to_county(-86,34))
print(coordinate_to_county(-97,42))
print(coordinate_to_county(-110,35))

('01055', 'Alabama', 'Etowah')
('31039', 'Nebraska', 'Cuming')
('04017', 'Arizona', 'Navajo')


In [129]:
# The following cell generates an interactive map of counties, coloured based on log_population. It took me ~30s to run this cell.

year = 2023
population_year_df = population_df[population_df['year'] == year]
gdf_population_year = pd.merge(counties_gdf, population_year_df, how='left', on='fips')

#gdf_population_year.explore("log_population", legend=True)


In [8]:
from shapely.geometry import Point


In [63]:
population_year_df = population_df.query('year == 2023')
gdf_population_year = pd.merge(counties_gdf, population_year_df, how='left', on='fips')
gdf_population_year.sample(10)

,STATEFP,COUNTYFP,COUNTYNS,fips,GEOIDFQ,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,...,INTPTLAT,INTPTLON,geometry,county,state,year,population,latitude,longitude,log_population
2124,01,089,00161570,01089,0500000US01089,Madison,Madison County,06,H1,G4020,...,+34.7642377,-086.5510802,"POLYGON ((-86.33524 34.61607, -86.33487 34.615...",Madison,Alabama,2023.0,412600.0,34.764,-86.551,5.615529
2109,41,065,01155136,41065,0500000US41065,Wasco,Wasco County,06,H1,G4020,...,+45.1645359,-121.1650693,"POLYGON ((-121.48217 45.51960, -121.47782 45.5...",Wasco,Oregon,2023.0,26333.0,45.165,-121.165,4.420500
2592,29,121,00758514,29121,0500000US29121,Macon,Macon County,06,H1,G4020,...,+39.8297950,-092.5643400,"POLYGON ((-92.84963 39.96288, -92.84960 39.964...",Macon,Missouri,2023.0,15161.0,39.830,-92.564,4.180728
1136,31,139,00835891,31139,0500000US31139,Pierce,Pierce County,06,H1,G4020,...,+42.2714129,-097.6109917,"POLYGON ((-97.48483 42.09068, -97.48513 42.090...",Pierce,Nebraska,2023.0,7299.0,42.271,-97.611,3.863263
596,16,077,00394951,16077,0500000US16077,Power,Power County,06,H1,G4020,...,+42.6941260,-112.8444070,"POLYGON ((-112.70462 42.32890, -112.70656 42.3...",Power,Idaho,2023.0,8253.0,42.694,-112.844,3.916612
315,05,015,00066975,05015,0500000US05015,Carroll,Carroll County,06,H1,G4020,...,+36.3373704,-093.5409718,"POLYGON ((-93.89038 36.37826, -93.89035 36.378...",Carroll,Arkansas,2023.0,28814.0,36.338,-93.542,4.459604
225,12,095,00295750,12095,0500000US12095,Orange,Orange County,06,H1,G4020,...,+28.5143906,-081.3232839,"POLYGON ((-81.15781 28.61222, -81.14608 28.612...",Orange,Florida,2023.0,1471416.0,28.514,-81.323,6.167735
3192,51,810,01498559,51810,0500000US51810,Virginia Beach,Virginia Beach city,25,C7,G4020,...,+36.7795254,-076.0291418,"POLYGON ((-76.22776 36.83385, -76.22686 36.834...",Virginia Beach,Virginia,2023.0,453649.0,36.779,-76.024,5.656720
1669,47,031,01639735,47031,0500000US47031,Coffee,Coffee County,06,H1,G4020,...,+35.4887654,-086.0782018,"POLYGON ((-85.93751 35.56678, -85.93748 35.566...",Coffee,Tennessee,2023.0,60633.0,35.489,-86.078,4.782709
1308,20,191,00481812,20191,0500000US20191,Sumner,Sumner County,06,H1,G4020,...,+37.2366617,-097.4933519,"POLYGON ((-97.47287 36.99869, -97.47492 36.998...",Sumner,Kansas,2023.0,22334.0,37.237,-97.493,4.348967


In [76]:
def coordinate_to_county(long,lat):
    """
    Input:
    lat: latitude
    long: longitude
    Output:
    FIPS, county name in 2023, state
    """
    coord = Point(long,lat)
    idx = gdf_population_year.geometry.contains(coord)
    res = gdf_population_year[idx]
    return res['fips'][0], res['state'][0], res['county'][0]

In [78]:
coordinate_to_county(-90, 41)

KeyError: 0